NOTE: This notebook has been designed to run on my local machine and not on google colab because it only uses the CPU, and the CPU on my PC is faster than the one on the colab VMs.

Imports

In [ ]:
import musdb
import museval
import multiprocessing
import functools
from pathlib import Path

Paths

In [ ]:
'''
These are the paths on my machine to the musdb dataset and the estimates 
computed by my neural network.
'''
mudDB_path = Path("~/Google Drive/musdb").expanduser()
estimates_path = Path("~/Google Drive/estimates-l2").expanduser()

Evaluation

In [ ]:
'''
For the evaluation I used the museval package, but I've done some changes to
parallelize the code and thus reduce the time needed.
'''

def evaluate(
  estimate: enumerate, 
  dataset: musdb.DB,
  tracks_to_be_estimated: list, 
  estimates_dir: str, 
  output_dir: str
) -> None:
  i, track = estimate
  print('%d/%d' % (i + 1, len(dataset)))
  if track.name in tracks_to_be_estimated:
    museval._load_track_estimates(
      track=track,
      estimates_dir=estimates_dir,
      output_dir=output_dir
    )

def parallel_eval_mus_dir(
  dataset,
  estimates_dir,
  output_dir=None,
  ext='wav',
  cores=4
):
  est = musdb.DB(root=estimates_dir, is_wav=True)
  tracks_to_be_estimated = [t.name for t in est.tracks]

  pool = multiprocessing.Pool(cores)

  pool.map(
    func=functools.partial(
      evaluate, 
      dataset = dataset,
      tracks_to_be_estimated = tracks_to_be_estimated, 
      estimates_dir = estimates_dir, 
      output_dir = output_dir),
    iterable=enumerate(dataset),
    chunksize=1,
  )

In [ ]:
# This cell compares each estimate with the original tracks and saves the results
mus = musdb.DB(mudDB_path, subsets=["test"])

parallel_eval_mus_dir(
  dataset=mus,
  estimates_dir=estimates_path,
  output_dir=estimates_path
)

In [ ]:
# This cell computes the median over all the tracks
stats = museval.aggregate.EvalStore()
stats.add_eval_dir(estimates_path)
print(stats)

These are the results of the last training I did

```
Aggrated Scores (median over frames, median over tracks)
vocals          ==> SDR:   3.663  SIR:   7.438  ISR:   6.375  SAR:   4.876  
drums           ==> SDR:   5.077  SIR:   6.177  ISR:   9.519  SAR:   5.300  
bass            ==> SDR:   4.479  SIR:   7.467  ISR:   8.705  SAR:   5.896  
other           ==> SDR:   2.494  SIR:   2.662  ISR:   6.287  SAR:   5.064  
```

